<a href="https://colab.research.google.com/github/utyabia/1-notebook/blob/main/Assignment_Week_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df = pd.read_csv('/content/Bank Marketing dataset.txt')
print(df.isnull().sum())

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64


In [12]:
# Fill missing values
df['industry'] = df['industry'].fillna('NA')
df['employment_status'] = df['employment_status'].fillna('NA')
df['annual_income'] = df['annual_income'].fillna(0.0)
df['lead_source'] = df['lead_source'].fillna('NA')
df['location'] = df['location'].fillna('NA')

In [3]:
from sklearn.model_selection import train_test_split

df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)  # 0.25 * 0.8 = 0.2 of total

In [4]:
from sklearn.metrics import roc_auc_score

numerical = ['lead_score', 'number_of_courses_viewed', 'interaction_count', 'annual_income']
y_train = df_train['converted'].values

auc_scores = {}

for col in numerical:
    auc = roc_auc_score(y_train, df_train[col])
    if auc < 0.5:
        auc = roc_auc_score(y_train, -df_train[col])
    auc_scores[col] = auc

auc_scores

{'lead_score': np.float64(0.6144993577250176),
 'number_of_courses_viewed': np.float64(0.7635680590007088),
 'interaction_count': np.float64(0.738270176293409),
 'annual_income': np.float64(0.5519578313253012)}

In [10]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

train_dicts = df_train.drop(columns=['converted']).to_dict(orient='records')
val_dicts = df_val.drop(columns=['converted']).to_dict(orient='records')

dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict_proba(X_val)[:, 1]
auc = roc_auc_score(df_val['converted'], y_pred)
print(round(auc, 3))

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [13]:
from sklearn.model_selection import train_test_split

df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)  # 0.25 * 0.8 = 0.2 of total

In [14]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

train_dicts = df_train.drop(columns=['converted']).to_dict(orient='records')
val_dicts = df_val.drop(columns=['converted']).to_dict(orient='records')

dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict_proba(X_val)[:, 1]
auc = roc_auc_score(df_val['converted'], y_pred)
print(f'AUC on validation set: {round(auc, 3)}')

AUC on validation set: 0.817


In [15]:
test_dicts = df_test.drop(columns=['converted']).to_dict(orient='records')
X_test = dv.transform(test_dicts)

y_pred_test = model.predict_proba(X_test)[:, 1]
auc_test = roc_auc_score(df_test['converted'], y_pred_test)
print(f'AUC on test set: {round(auc_test, 3)}')

AUC on test set: 0.82


In [11]:
print("Missing values in df_train:")
print(df_train.isnull().sum())
print("\nMissing values in df_val:")
print(df_val.isnull().sum())
print("\nMissing values in df_test:")
print(df_test.isnull().sum())

Missing values in df_train:
lead_source                 75
industry                     0
number_of_courses_viewed     0
annual_income                0
employment_status            0
location                    37
interaction_count            0
lead_score                   0
converted                    0
dtype: int64

Missing values in df_val:
lead_source                 22
industry                     0
number_of_courses_viewed     0
annual_income                0
employment_status            0
location                    13
interaction_count            0
lead_score                   0
converted                    0
dtype: int64

Missing values in df_test:
lead_source                 31
industry                     0
number_of_courses_viewed     0
annual_income                0
employment_status            0
location                    13
interaction_count            0
lead_score                   0
converted                    0
dtype: int64


In [17]:
# Q3: Precision-Recall intersection
import numpy as np
from sklearn.metrics import precision_score, recall_score

thresholds = np.linspace(0, 1, 101)
precisions = []
recalls = []

y_val_true = df_val['converted'].values
y_pred_val = model.predict_proba(X_val)[:, 1] # Get validation predictions

for t in thresholds:
    y_pred_bin = (y_pred_val >= t).astype(int)
    precisions.append(precision_score(y_val_true, y_pred_bin, zero_division=1))
    recalls.append(recall_score(y_val_true, y_pred_bin))

# Find intersection
intersection_threshold = None
for i, t in enumerate(thresholds):
    if abs(precisions[i] - recalls[i]) < 0.005:  # small tolerance
        intersection_threshold = t
        break

print(f"Q3 - Precision/Recall intersection at threshold: {intersection_threshold:.3f}")

Q3 - Precision/Recall intersection at threshold: 0.640


In [19]:
# Q4: Max F1 threshold
from sklearn.metrics import f1_score

f1_scores = []
for t in thresholds:
    y_pred_bin = (y_pred_val >= t).astype(int)
    f1_scores.append(f1_score(y_val_true, y_pred_bin))

best_idx = np.argmax(f1_scores)
best_threshold = thresholds[best_idx]
print(f"Q4 - Best F1 at threshold: {best_threshold:.3f}")

Q4 - Best F1 at threshold: 0.570


In [21]:
# Q5: 5-Fold CV
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=1)
scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_fold_train = df_full_train.iloc[train_idx]
    df_fold_val = df_full_train.iloc[val_idx]

    train_dicts = df_fold_train.to_dict(orient='records')
    val_dicts = df_fold_val.to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_fold_train = dv.fit_transform(train_dicts)
    X_fold_val = dv.transform(val_dicts)

    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)
    model.fit(X_fold_train, df_fold_train['converted'])

    y_pred_fold = model.predict_proba(X_fold_val)[:, 1]
    auc = roc_auc_score(df_fold_val['converted'], y_pred_fold)
    scores.append(auc)

print(f"Q5 - CV Scores: {np.round(scores, 4)}")
print(f"Mean: {np.mean(scores):.3f}, Std: {np.std(scores):.4f}")

Q5 - CV Scores: [0.8688 0.9217 0.8434 0.8422 0.9026]
Mean: 0.876, Std: 0.0318


In [22]:
# Q6: Hyperparameter tuning
C_values = [0.000001, 0.001, 1]
results = []

for C in C_values:
    kfold = KFold(n_splits=5, shuffle=True, random_state=1)
    scores = []

    for train_idx, val_idx in kfold.split(df_full_train):
        df_fold_train = df_full_train.iloc[train_idx]
        df_fold_val = df_full_train.iloc[val_idx]

        train_dicts = df_fold_train.to_dict(orient='records')
        val_dicts = df_fold_val.to_dict(orient='records')

        dv = DictVectorizer(sparse=False)
        X_fold_train = dv.fit_transform(train_dicts)
        X_fold_val = dv.transform(val_dicts)

        model = LogisticRegression(solver='liblinear', C=C, max_iter=1000)
        model.fit(X_fold_train, df_fold_train['converted'])

        y_pred_fold = model.predict_proba(X_fold_val)[:, 1]
        auc = roc_auc_score(df_fold_val['converted'], y_pred_fold)
        scores.append(auc)

    mean_score = np.mean(scores)
    std_score = np.std(scores)
    results.append((C, mean_score, std_score))
    print(f"C={C}: Mean={mean_score:.3f}, Std={std_score:.3f}")

# Find best C
best_C = min(results, key=lambda x: (-x[1], x[2], x[0]))[0]  # max mean, min std, min C
print(f"Q6 - Best C: {best_C}")

C=1e-06: Mean=0.561, Std=0.024
C=0.001: Mean=0.926, Std=0.018
C=1: Mean=0.876, Std=0.032
Q6 - Best C: 0.001
